In [ ]:
import numpy as np
import cma
import cma.purecma as purecma
import matplotlib.pyplot as plt

# Formes Canoniques d'Algorithmes Evolutionnaires

## Nomenclature générale d'un Algorithme Evolutionnaire : 
Un algorithme evolutionnaire reprend géneralement les composantes suivantes : 
* Une population d'individus (un ou plusieurs) representant des génotypes 
* Une méthode de perturbation des individus pour générer des nouveaux individus (successeurs ou offspring)
* Une fonction d'évaluation qui sert a comparer les genotypes en fonction de leur phenotypes et a en sélectionner certains.

Donc pour chaque élément les variantes les plus communes sont : 
- Pour la population : 1 Si elle se compose d'un seul élement $\mu$ si plusieurs et $\mu / \sigma$ si parmi la population un nombre $\sigma$ seulement participent a la créations des successeurs.
- Successeurs : 1 ou $\lambda$ si plusieurs successeurs.
- Mode de sélection "," si la sélection se fait parmi l'offspring ou "+" si elle se fait parmi les parents et l'offspring.

Donc par exemple : 
- (1,1) ne peut pas exister ça voudrait dire qu'on sélectionne parmi une offspring de 1 donc enfaite le nouvel élément serrait seul et on n'aurait pas avec qui le comparer on le prendrait systématiquement
- (1+1) signifie qu'on a genere un individu, qu'on perturbe a chaque étape et qu'on garde l'offspring si elle est meilleur que l'individu (donc on effectue une sélection sur le parent et l'offspring).
- (1,$\lambda$) veut dire que a chaque étape on génère plusieures descendants et on remplace le parent systématiquement par le meilleur élément de l'offspring (le parent ne participe pas a la sélection).
- ($\mu$ + $\lambda$) veut dire que la sélection de la prochaine population qui contient plusieurs individus se fait parmi plusieurs l'offspring et l'ancienne population. 

## Algorithmes evolutionnaires basiques
Dans cette partie nous allons implémenter et comparer différents algorithmes génétiques simples en les testant sur le problème du Lab 01 sur la prédiction des prix des maisons dans boston.

In [ ]:
"""
On commence par l'algorithme le plus simple qu'on puisse imaginer, un (1+1) avec perturbation normale.
"""
def launch_oneplusone(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i=0
    j=0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    while i<nbeval:
        child = parent + sigma*np.random.normal(0,np.ones(parent.shape),parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        j+=1
        solutions = np.array([parent])
        i+=1
    print ("Best fit",bestFit,"at iteration",bestIt)
    return bestFit


In [ ]:
"""
Dans ce qui suit nous utiliserons la structure de réseaux de neurones simple que nous avons vu précédement
car elle permet de manipuler les poids de façon "flat" c'est a dire en amount (de façon non structurée)
(Pas la peine de relire tout ça c'est pas important, c'est a titre indicatif et on aurait pu utiliser scikit-learn
remarquez juste que j'ai mis une fonction d'activation linéaire a la place de la sigmoid)
"""
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            #y = sigmoid(a)
            y = a
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            """
            ICI !! 
            """
            #out = tanh(a)
            out = a
            return out
        else: # Simple monolayer perceptron
            """
            et ICI !! 
            """
            #return tanh(np.matmul(x,self.weights[0]) + self.bias[0])
            return np.matmul(x,self.weights[0]) + self.bias[0]

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
#On charge les données 
#On utilise un réseau de neurones simple 
Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
print(Network.get_parameters())

In [ ]:
#La fonction d'evalutation qu'on prend va associer a chaque vecteur de poids w l'erreur moyenne quadratique

def evaluation_function(w):
    X, y = load_boston(return_X_y=True)
    Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
    Network.set_parameters(w)
    y_pred = Network.predict(X)
    return mean_squared_error(y,y_pred)

Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
init_point = Network.get_parameters()
evaluation_function(init_point)


In [ ]:
launch_oneplusone(init_point, 0.01, evaluation_function, nbeval=10000)

In [ ]:
"""
Petit rafinement de l'algorithme en ajoutant la règle des 1/5.
Remarque : Ce rafinement contribue a réduire la stochacité de l'algorithme et donc sont utilisation doit être
bien mesurée car elle le rend trés dépendant du point de départ
"""

def launch_ESonefifthRule(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            sigma = sigma*2
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        else:
            sigma = sigma*(2**(-(1/4)))
        j += 1
        solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit


launch_ESonefifthRule(init_point, 0.1, evaluation_function, nbeval=10000)

In [ ]:
def launch_LambdaES(center, _lambda, sigma,ma_func, nbeval=10000):
    center = np.array(center)
    parent = center + sigma * np.random.normal(0, center.shape, center.shape[0])
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        for k in range(_lambda):
            child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
            childFit = ma_func(child)
            if childFit <= parentFit:
                parentFit = childFit
                parent = np.copy(child)
                sigma = sigma * 2
                if bestFit > parentFit:
                    bestFit = parentFit
                    bestIt = i
                    print("New best fit found : ", bestFit," a l'itteration : ",i)
            else:
                sigma = sigma * (2 ** (-(1 / 4)))
            j += 1
            solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit

launch_LambdaES(init_point,100, 0.01, evaluation_function, nbeval=1000)

In [ ]:
res = cma.CMAEvolutionStrategy(init_point, 0.01).optimize(evaluation_function,maxfun=10000).result


In [ ]:
res = purecma.fmin(evaluation_function,init_point,0.01,maxfevals=10000)